# Práctica de Bioinformática: Análisis Estructural de Proteínas

* Asignatura: Bioinformática  
* Laboratorio: Sesión 06 - Estructuras Tridimensionales  

---

### Integrantes del equipo:
* Adrián Ojeda
* Raúl Mendoza

---

### Introducción
En este jupyter notebook utilizaremos la librería Biopython para analizar estructuras tridimensionales de proteínas. Se abordarán tareas fundamentales como:
1.  Cálculo de distancias atómicas y ángulos diedros.
2.  Determinación del centro de masas.
3.  Alineamiento de secuencias y estructuras.
4.  Análisis filogenético básico.

Para ello, trabajaremos con proteínas reales obtenidas del Protein Data Bank (PDB), como la Hemoglobina y la Lisozima, así como neuropéptidos (Hipocretinas/Orexinas).

In [5]:
# Instalación de librerías necesarias (si no tienes instalados la librerías necesarias en tu entorno, descomentar la siguiente línea)
# !pip install biopython numpy

import sys
import numpy as np
import warnings

from Bio.PDB import PDBList, MMCIFParser
from Bio.PDB.vectors import calc_dihedral, Vector

warnings.filterwarnings('ignore')

print(f"Librerías importadas correctamente. Versión de Python: {sys.version.split()[0]}")

Librerías importadas correctamente. Versión de Python: 3.13.1


## Ejercicio 1: La Hemoglobina Humana (1A3N)

* Objetivo: Descargar la estructura de la proteína de la hemoglobina humana (código PDB: 1A3N), visualizarla conceptualmente mediante código y realizar cálculos geométricos.

La hemoglobina es una hemoproteína de la sangre que transporta oxígeno. En este ejercicio realizaremos las siguientes operaciones:

1.  (a) Calcular la **distancia euclidiana** entre los átomos de Oxígeno (O) del primer y último residuo de la cadena A.
    * Fórmula de distancia: $d(p, q) = \sqrt{(q_1 - p_1)^2 + (q_2 - p_2)^2 + (q_3 - p_3)^2}$
2.  (b) Calcular el **ángulo diedro** ($\phi$ o $\psi$) formado por los átomos N, CA, C y O del primer residuo de la cadena A.
3.  (c) Calcular el **centro de masas** ($R$) de toda la estructura.
    * $R = \frac{1}{M} \sum_{i=1}^{n} m_i \mathbf{r}_i$

In [6]:
pdb_id = "1A3N"
pdbl = PDBList()
file_path = pdbl.retrieve_pdb_file(pdb_id, pdir='.', file_format='mmCif')
parser = MMCIFParser()
structure = parser.get_structure(pdb_id, file_path)

print(f"\n Estructura {pdb_id} cargada exitosamente.")

Structure exists: '.\1a3n.cif' 

 Estructura 1A3N cargada exitosamente.


In [7]:
model = structure[0]
chain_a = model['A']
residues = [res for res in chain_a if res.id[0] == ' ']
first_res = residues[0]
last_res = residues[-1]

print(f"\n- Apartado (a): Distancia")
print(f"Primer residuo: {first_res.get_resname()} (ID: {first_res.id[1]})")
print(f"Último residuo: {last_res.get_resname()} (ID: {last_res.id[1]})")

if 'O' in first_res and 'O' in last_res:
    atom_o_first = first_res['O']
    atom_o_last = last_res['O']
    
    distance = atom_o_first - atom_o_last
    print(f"Distancia entre átomos O: {distance:.4f} Å")
else:
    print("Error: No se encontraron átomos de oxígeno en los residuos extremos.")


- Apartado (a): Distancia
Primer residuo: VAL (ID: 1)
Último residuo: ARG (ID: 141)
Distancia entre átomos O: 22.2284 Å


In [8]:
print(f"\n- Apartado (b): Ángulo Diedro")
try:
    n = first_res['N']
    ca = first_res['CA']
    c = first_res['C']
    o = first_res['O']
    
    vector_n = n.get_vector()
    vector_ca = ca.get_vector()
    vector_c = c.get_vector()
    vector_o = o.get_vector()
    
    angle_rad = calc_dihedral(vector_n, vector_ca, vector_c, vector_o)
    angle_deg = np.degrees(angle_rad)

    print(f"Átomes usados: N, CA, C, O de {first_res.get_resname()}")
    print(f"Ángulo diedro: {angle_deg:.4f} grados")
    
except KeyError as e:
    print(f"No se pudo calcular el diedro, falta el átomo: {e}")


- Apartado (b): Ángulo Diedro
Átomes usados: N, CA, C, O de VAL
Ángulo diedro: -25.6655 grados


In [9]:
print(f"\n- Apartado (c): Centro de Masas")

def calculate_center_of_mass(entity):
    atomic_masses = {"H": 1.008, "C": 12.011, "N": 14.007, "O": 15.999, "S": 32.065, "P": 30.974, "FE": 55.845}
    total_mass = 0.0
    center_of_mass = np.array([0.0, 0.0, 0.0])
    
    for atom in entity.get_atoms():
        element = atom.element.upper().strip()
        mass = atomic_masses.get(element, 12.01) 
        
        pos = atom.get_coord()
        center_of_mass += pos * mass
        total_mass += mass
        
    return center_of_mass / total_mass

com = calculate_center_of_mass(structure)
print(f"Centro de masas (x, y, z):")
print(f"[{com[0]:.4f}, {com[1]:.4f}, {com[2]:.4f}]")


- Apartado (c): Centro de Masas
Centro de masas (x, y, z):
[14.4469, 2.0080, 13.1803]
